In [1]:
import torch
import cv2

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5m')

# S -> tiki-taka : 68 fps
# M -> tiki-taka : 60 fps

Using cache found in /home/nath_intern/.cache/torch/hub/ultralytics_yolov5_master

                 from  n    params  module                                  arguments                     
  0                -1  1      5280  models.common.Focus                     [3, 48, 3]                    
  1                -1  1     41664  models.common.Conv                      [48, 96, 3, 2]                
  2                -1  1     65280  models.common.C3                        [96, 96, 2]                   
  3                -1  1    166272  models.common.Conv                      [96, 192, 3, 2]               
  4                -1  1    629760  models.common.C3                        [192, 192, 6]                 
  5                -1  1    664320  models.common.Conv                      [192, 384, 3, 2]              
  6                -1  1   2512896  models.common.C3                        [384, 384, 6]                 
  7                -1  1   2655744  models.common.Conv       

Adding autoShape... 


In [3]:
model.to(torch.device('cpu'));

In [4]:
#model.half();

In [5]:
next(model.parameters()).is_cuda

False

In [6]:
#model.classes = [0]

In [7]:
import time
from IPython.display import clear_output

torch.backends.cudnn.benchmark = False

video_path = 'videos/kbank-cctv-1.mp4'
out_video_path = 'visual_result/kbank-cctv-2.mp4'
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

count = 0
n_frame_skip = 1

videoWriter = cv2.VideoWriter(out_video_path, fourcc, fps, size)

In [8]:
print(f'FPS: {fps}, Size: {size}')

FPS: 30.06376126226387, Size: (1376, 776)


In [9]:
FONT_SCALE = 1
FONT_FACE = cv2.FONT_HERSHEY_SIMPLEX
FONT_COLOR = (39, 0, 247)
FONT_THICKNESS = 3

In [ ]:
torch.backends.cudnn.benchmark = False

while (cap.isOpened()):
    flag, img = cap.read()
    if not flag:
        break
    s1 = time.time()
    
    results = model(img, size=400)
    results.imgs
    results.render()
    vis_img = results.imgs[0]
    
    
    fps_process = 1 / (time.time()-s1)
    
    cv2.putText(vis_img,f'FPS {fps_process:.2f}', (size[0]-200, 30),  FONT_FACE, FONT_SCALE, FONT_COLOR, FONT_THICKNESS)    
    cv2.imshow('Image', vis_img)
    
    #if save_out_video:
    videoWriter.write(vis_img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    # Frame Skipping
    #count += n_frame_skip
    #cap.set(1, count)

cap.release()
videoWriter.release()
    
cv2.destroyAllWindows()